# Download Sentinel-2 data for ASO rasters

Given an ASO raster, find Sentinel-2 scenes over the same area within a week of lidar acqusition, then choose scene with most snow pixels (find max coverage area and least cloud pixels).

In [1]:
# based on exmaples from
# https://planetarycomputer.microsoft.com/docs/tutorials/cloudless-mosaic-sentinel2/
# https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
import glob
import rioxarray as rxr
import re
import datetime
import pandas as pd
from shapely.geometry import box
import odc.stac

In [2]:
def sentinel2_for_aso(aso_raster_fn):
    
    time = pd.to_datetime(re.search("(\d{4}\d{2}\d{2})", aso_raster_fn).group())
    week_before = (time - datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
    week_after = (time + datetime.timedelta(weeks=1)).strftime('%Y-%m-%d')
    time_of_interest = f'{week_before}/{week_after}'
    
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    
    
    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

    search = catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=bounds_latlon,
        datetime=time_of_interest)

    # Check how many items were returned
    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    
    sentinel2_stac = odc.stac.load(items,chunks={"x": 2048, "y": 2048},resolution=60)
    sentinel2_stac_clipped = sentinel2_stac.rio.clip_box(*bounds_latlon.bounds,crs="EPSG:4326")
    
    scl = sentinel2_stac_clipped['SCL'].rio.reproject_match(aso_raster).where(aso_raster>0)
    
    classes = [ #SCL classes here: https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/scene-classification/
    #0,   #No Data (Missing data)	#000000	
    #1,   #Saturated or defective pixel	#ff0000	
    #2,   #Topographic casted shadows (called "Dark features/Shadows" for data before 2022-01-25)	#2f2f2f	
    #3,   #Cloud shadows	#643200	
    4,   #Vegetation	#00a000	
    5,   #Not-vegetated	#ffe65a	
    #6,   #Water	#0000ff	
    #7,   #Unclassified	#808080	
    #8,   #Cloud medium probability	#c0c0c0	
    #9,   #Cloud high probability	#ffffff	
    #10,   #Thin cirrus	#64c8ff	
    11    #Snow or ice      
    ]
    
    idx_least_clouds = scl.where(scl.isin(classes)).sum(dim=['x','y']).idxmax()
    sentinel2_best_lowcloud = sentinel2_stac_clipped.sel(time=idx_least_clouds)
    
    sentinel2_best_lowcloud.to_netcdf(f'sentinel2_data/S2_{pd.to_datetime(idx_least_clouds.values).strftime("%Y%m%d")}_for_{aso_raster_fn[8:-4]}.nc')

In [3]:
aso_raster_fns = glob.glob('rasters/*')

In [4]:
for aso_raster_fn in aso_raster_fns:
    sentinel2_for_aso(aso_raster_fn)
    print(f'{aso_raster_fn} complete.')

Returned 15 Items


/srv/conda/envs/notebook/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


rasters/ASO_50M_SD_GreenRiver_20220611_clean.tif complete.
